In [1]:
from getpass import getpass
import requests
import json
from datetime import datetime, timedelta
from time import sleep
import numpy as np
import pandas as pd
import os
import csv
import datetime
import dateutil.parser
import unicodedata
import time

In [2]:
bearer_token = getpass('Enter Bearer Token: ')

Enter Bearer Token: ··········


In [3]:
def create_headers(bearer_token):
  headers = {"Authorization": "Bearer {}".format(bearer_token)}
  return headers

In [4]:
def create_url(keyword, start_date, end_date, max_results=10):
  search_url = 'https://api.twitter.com/2/tweets/search/all'

  query_params = {'query': keyword,
                  'start_time': start_date,
                  'end_time': end_date,
                  'max_results': max_results,
                  'tweet.fields': 'id,text,created_at',
                  'place.fields': 'full_name',
                  'expansions': 'geo.place_id',
                  'next_token': {}}

  return (search_url, query_params)

In [5]:
def connect_to_endpoint(url, headers, params, next_token = None):
  params['next_token'] = next_token
  response = requests.request("GET", url, headers = headers, params = params)
  print("Endpoint Response Code: " + str(response.status_code))
  
  if response.status_code != 200:
    raise Exception(response.status_code, response.text)
  
  return response.json()

In [6]:
def append_to_csv(json_response, fileName):
  counter = 0
  csvFile = open(filename, 'a', newline='', encoding='utf-8')
  csvWriter = csv.writer(csvFile)

  for tweet in json_response['data']:
    id = tweet['id']
    created_at = dateutil.parser.parse(tweet['created_at'])
    text = tweet['text']
    
    if ('geo' in tweet):   
        geo = tweet['geo']['place_id']
    else:
        geo = ' '
    
    res = [id, created_at, geo, text]
    csvWriter.writerow(res)
    counter += 1

  csvFile.close()
  print('{} of Tweets added from this response: '.format(counter))

In [7]:
filename = '/content/drive/MyDrive/Doutorado/Artigos/inflacao-twitter/data/' \
            + datetime.datetime.today().strftime('%Y%m%d_%H%M%S') \
            + '.csv'

headers = create_headers(bearer_token)

keyword = 'inflacao lang:pt place_country:BR'
max_results = 500

date_list = pd.date_range(start='2006-10-01', end='2021-09-30') #2006-10-01
start_list = [x.strftime('%Y-%m-%dT00:00:00.000Z') for x in date_list]
end_list = [x.strftime('%Y-%m-%dT23:59:59.999Z') for x in date_list]

In [14]:
start_list[4634]

'2019-06-09T00:00:00.000Z'

In [15]:
csvFile = open(filename, 'a', newline='', encoding='utf-8')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['id', 'created_at', 'geo', 'text'])
csvWriter.writerow([keyword, '0', '0', '0'])
csvFile.close()

In [16]:
total_tweets = 0

for i in range(4634, len(start_list)):
  count = 0
  flag = True
  next_token = None
  
  while flag:
    print('-------------------')
    print('Token: ', next_token)
    url = create_url(keyword, start_list[i], end_list[i], max_results)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
      next_token = json_response['meta']['next_token']
      print('Next Token: ', next_token)
      if result_count is not None and result_count > 0 and next_token is not None:
        print('Start Date: ', start_list[i])
        append_to_csv(json_response, filename)
        count += result_count
        total_tweets += result_count
        print('Total # of Tweets added: ', total_tweets)
        print('-------------------')
        time.sleep(3)
    else:
      if result_count is not None and result_count > 0:
        print('-------------------')
        print('Start Date: ', start_list[i])
        append_to_csv(json_response, filename)
        count += result_count
        total_tweets += result_count
        print('Total # of Tweets added: ', total_tweets)
        print('-------------------')
        time.sleep(3)
      flag = False
      next_token = None
      time.sleep(3)

A saída de streaming foi truncada nas últimas 5000 linhas.
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-01-15T00:00:00.000Z
21 of Tweets added from this response: 
Total # of Tweets added:  2837
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-01-16T00:00:00.000Z
12 of Tweets added from this response: 
Total # of Tweets added:  2849
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-01-17T00:00:00.000Z
13 of Tweets added from this response: 
Total # of Tweets added:  2862
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-01-18T00:00:00.000Z
9 of Tweets added from this response: 
Total # of Tweets added:  2871
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-01-